In [3]:
# Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib


In [4]:
# Create the output file 
dataDir = '/Users/student/PycharmProjects/data'

subjects = os.listdir(dataDir + '/raw/')
subjects = sorted(subjects)[::]

# List of task configs
TaskConfigs = ['A1_forw','A1_backw',
               'B1_forw','B1_backw',
               'C1_forw','C1_backw',
               'D1_forw','D1_backw',
               'E1_forw','E1_backw',
               'A2_forw','A2_backw',
               'B2_forw','B2_backw',
               'C2_forw','C2_backw',
               'D2_forw','D2_backw',
               'E2_forw','E2_backw']
# Create list of the configs tested
data = {
'TaskConfigs' : TaskConfigs * 5,
'Repeat' : [i for i in range(0, 5) for _ in range(20)]
}

executionPeriods = pd.DataFrame(data)

for subject in subjects:
    executionPeriods[subject + '_start' ] = np.nan
    executionPeriods[subject + '_end'   ] = np.nan

executionPeriods

,TaskConfigs,Repeat,sub-01_start,sub-01_end,sub-02_start,sub-02_end,sub-03_start,sub-03_end,sub-04_start,sub-04_end,...,sub-31_start,sub-31_end,sub-32_start,sub-32_end,sub-33_start,sub-33_end,sub-34_start,sub-34_end,sub-35_start,sub-35_end
0,A1_forw,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A1_backw,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B1_forw,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B1_backw,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C1_forw,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,C2_backw,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,D2_forw,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,D2_backw,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,E2_forw,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# set hyperparameters for analysis
subject = 'sub-02'
task_half = 1
repeat = 4
Counter = 0
subjects = [
    # 'sub-01',
    'sub-02', 
    'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 
    # 'sub-10', 
    'sub-11', 'sub-12', 'sub-13', 'sub-14', 'sub-15', 'sub-16', 'sub-17', 'sub-18', 'sub-19', 'sub-20', 
    # 'sub-21',
    'sub-22', 'sub-23', 'sub-24', 'sub-25', 'sub-26', 'sub-27', 'sub-28',
    'sub-29', 'sub-30', 'sub-31', 'sub-32', 'sub-33', 'sub-34', 'sub-35'
          ]

for subject in subjects:

    for task_half in [1, 2]:

        for repeat in range(5):
            
            end_time = 0
            start_time = 0
            # Load in the correct behaviour file for each subject and task half
            behaviourFile = pd.read_csv(dataDir + '/raw/' + subject + '/beh/' + f"{subject}_fmri_pt{task_half}.csv")
            # Forward fill the 'task_config' column
            behaviourFile['task_config'] = behaviourFile['task_config'].ffill()
            # Add a "5" to the end final value of the 'rep_runs.thisN' column
            # final index of the 'rep_runs.thisN' column 
            final_index = behaviourFile['rep_runs.thisN'].last_valid_index()
            # Add a 5 to the final value of the 'rep_runs.thisN' column
            behaviourFile.at[final_index, 'rep_runs.thisN'] = 5
            # Get the list of task configs that were tested in this behaviour file
            # List of task configs, list of directions (everything is repeeated 5 times)
            taskConfigs = behaviourFile['TaskConfigs'] = behaviourFile['task_config'] + '_' + behaviourFile['type']
            taskConfigs = behaviourFile['TaskConfigs'].unique()
            taskConfigs = taskConfigs[~pd.isnull(taskConfigs)]

            # For each of theses configs, get the start and end time of the execution period
            for taskConfig in list(taskConfigs):
                # print(subject, task_half, repeat, taskConfig)
                task, direction = taskConfig.split('_')
                # get the rows of the behaviour file that correspond to this task and direction
                task_behaviour = behaviourFile[(behaviourFile['task_config'] == task) & (behaviourFile['type'] == direction)]

                # start time is the time of the first ABCD screen
                start_time = task_behaviour['start_ABCD_screen'].dropna().iloc[repeat]

                # print(start_time)

                # end time is the 
                if repeat == 4:
                    # If it is the final repeat

                    finalRepeatRow = behaviourFile[(behaviourFile['task_config'] == task) & (behaviourFile['type'] == direction) & (behaviourFile['rep_runs.thisN'] == 5)]

                    # print(finalRepeatRow)
                    end_time = finalRepeatRow['t_step_end_global'].iloc[0]
                    pass
                else:
                #     # If it is not the final repeat
                    end_time = task_behaviour['start_ABCD_screen'].dropna().iloc[repeat + 1]
                    pass
                # print(end_time)


                # Fill in the executionPeriods dataframe
                # Fill in the correct values in the executionPeriods dataframe
                # Get the correct executionPeriods df index
                index = executionPeriods[(executionPeriods['TaskConfigs'] == taskConfig) & (executionPeriods['Repeat'] == repeat)].index[0]

                # Fill in the correct values
                executionPeriods.at[index, subject + '_start']  = start_time
                executionPeriods.at[index, subject + '_end']    = end_time
        # break
    # break
    # Save the executionPeriods dataframe to a csv file

executionPeriods

,TaskConfigs,Repeat,sub-01_start,sub-01_end,sub-02_start,sub-02_end,sub-03_start,sub-03_end,sub-04_start,sub-04_end,...,sub-31_start,sub-31_end,sub-32_start,sub-32_end,sub-33_start,sub-33_end,sub-34_start,sub-34_end,sub-35_start,sub-35_end
0,A1_forw,0,NaN,NaN,1419.135426,1443.860446,1352.744528,1373.033785,683.758982,703.662297,...,802.864800,837.019536,770.147640,798.767995,1226.246341,1251.932167,1653.770664,1683.574194,1190.560024,1229.168560
1,A1_backw,0,NaN,NaN,965.864790,1000.192647,73.505212,97.519862,187.857300,220.370459,...,954.714682,988.359446,1086.992082,1112.200248,572.011047,599.034666,1140.041949,1171.086904,371.263808,399.083553
2,B1_forw,0,NaN,NaN,1119.795621,1154.163113,849.013000,881.995843,825.299731,855.217214,...,1594.724406,1621.653649,1236.904813,1270.056757,45.751387,86.935409,1475.063728,1514.970232,1510.669845,1544.009519
3,B1_backw,0,NaN,NaN,484.309799,520.161907,1491.464480,1517.305786,1134.530774,1159.310798,...,1268.051444,1296.428282,220.926317,263.640059,1541.359351,1571.903983,397.450750,443.296014,1023.015611,1048.233772
4,C1_forw,0,NaN,NaN,340.341433,367.223678,1015.042485,1046.130856,983.031146,1008.744931,...,637.685918,673.999140,1420.716194,1456.249385,1066.316731,1100.718455,958.401018,997.058094,83.744485,107.442436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,C2_backw,4,NaN,NaN,1003.761884,1028.309777,1631.853194,1656.070950,211.916284,252.495114,...,1009.809899,1039.787788,640.877747,661.470263,833.420339,857.504306,665.553463,695.239545,1143.232287,1167.250881
96,D2_forw,4,NaN,NaN,1435.644581,1463.565469,780.982588,803.135327,1106.526824,1127.473754,...,1699.881443,1727.861462,162.767400,186.095456,246.767823,275.344984,1316.291152,1338.560723,330.269159,351.819439
97,D2_backw,4,NaN,NaN,426.182985,449.135733,1284.544607,1305.936012,1249.532190,1271.992552,...,367.631558,394.117175,1131.210192,1156.289576,992.816701,1021.998903,1157.711152,1188.293564,804.977953,829.479317
98,E2_forw,4,NaN,NaN,581.296579,602.918029,322.167613,342.022546,1695.141778,1717.911712,...,188.049201,211.743871,983.206736,1009.889692,1882.069713,1907.424845,171.933539,200.834219,491.007063,517.466045


In [114]:
# save the execution periods to a csv file
executionPeriods.to_csv(dataDir + '/executionPeriods.csv', index = False)

In [12]:
# Convert to TRs
TR_LENGTH = 1.078 # seconds / TR
# Remove the offset 
for column in executionPeriods.columns:
    if 'start' in column or 'end' in column:
        # Convert to TRs from seconds
        executionPeriods[column] = executionPeriods[column] / TR_LENGTH
        # Round to the nearest TR and convert to int
        executionPeriods[column] = executionPeriods[column].round()
        # Convert to numeric, coerce errors, then convert to nullable integer type
        executionPeriods[column] = pd.to_numeric(executionPeriods[column], errors='coerce').astype('Int64')
        
        # Add the offset to the start time 
        sub_df = None
        # # Get the offset for this subject
        # FIRST_TR_TIME = sub_df['TR_received_no0'].dropna().unique().tolist()[0]
        # # Add the offset to the start time
        # executionPeriods[column] = executionPeriods[column] - FIRST_TR_TIME
executionPeriods


,TaskConfigs,Repeat,sub-01_start,sub-01_end,sub-02_start,sub-02_end,sub-03_start,sub-03_end,sub-04_start,sub-04_end,...,sub-31_start,sub-31_end,sub-32_start,sub-32_end,sub-33_start,sub-33_end,sub-34_start,sub-34_end,sub-35_start,sub-35_end
0,A1_forw,0,<NA>,<NA>,1316,1339,1255,1274,634,653,...,745,776,714,741,1138,1161,1534,1562,1104,1140
1,A1_backw,0,<NA>,<NA>,896,928,68,90,174,204,...,886,917,1008,1032,531,556,1058,1086,344,370
2,B1_forw,0,<NA>,<NA>,1039,1071,788,818,766,793,...,1479,1504,1147,1178,42,81,1368,1405,1401,1432
3,B1_backw,0,<NA>,<NA>,449,483,1384,1408,1052,1075,...,1176,1203,205,245,1430,1458,369,411,949,972
4,C1_forw,0,<NA>,<NA>,316,341,942,970,912,936,...,592,625,1318,1351,989,1021,889,925,78,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,C2_backw,4,<NA>,<NA>,931,954,1514,1536,197,234,...,937,965,595,614,773,795,617,645,1061,1083
96,D2_forw,4,<NA>,<NA>,1332,1358,724,745,1026,1046,...,1577,1603,151,173,229,255,1221,1242,306,326
97,D2_backw,4,<NA>,<NA>,395,417,1192,1211,1159,1180,...,341,366,1049,1073,921,948,1074,1102,747,769
98,E2_forw,4,<NA>,<NA>,539,559,299,317,1572,1594,...,174,196,912,937,1746,1769,159,186,455,480


## Convert the data from the whole RAW BOLD video to the specific subsets that match the exection
for each subject
    for each config (A1f, A1b, B1f, ...) (20x) (dir)
        <!-- for each repeat (1-5) (5x)
            file 
        OR   -->
        concat over repeats
        OR 
        oonly first repeat

        EV folder to check the values of the timings
        same as PEs

In [17]:
# Get each subjects eeecution period for each repeat


# Load in the exection Peroids dataframe
# executionPeriods = pd.read_csv(dataDir + '/executionPeriods.csv')

subjects = ['sub-02']
subject = subjects[0]
task_half = 1
# Configs
TaskConfigs = ['A1_forw','A1_backw',
               'B1_forw','B1_backw',
               'C1_forw','C1_backw',
               'D1_forw','D1_backw',
               'E1_forw','E1_backw',
               'A2_forw','A2_backw',
               'B2_forw','B2_backw',
               'C2_forw','C2_backw',
               'D2_forw','D2_backw',
               'E2_forw','E2_backw']
# 
Config = TaskConfigs[0]

for subject in subjects:
    
    # Load in the signal file
    filtered_func_file = nib.load(dataDir + f'/derivatives/{subject}/func/preproc_clean_0{task_half}.feat/filtered_func_data.nii.gz')
    
    for Config in TaskConfigs:
        # subset of the data that corresponds to the correct task config
        configDataframe = executionPeriods[executionPeriods['TaskConfigs'] == Config]
        # Get the correct pair of indicies from the df to define the start and end of the execution period
        for repeat in range(5): # there are 5 repeats
            start_TR, end_TR = configDataframe['sub-02_start'].iloc[repeat], configDataframe['sub-02_end'].iloc[repeat]
            # Get the correct signal bits of the BOLD file

            # If its the whole execution period will be the concantentation of each repeat's execution period. 

            filtered_func_data = filtered_func_file.get_fdata()
            # Get the signal for the correct period
            print(start_TR, end_TR) # these must be integers not floats
            condition = filtered_func_data[..., start_TR:end_TR]

            # save the the correct directory / subdirectory structure
            folder = f"/derivatives/{subject}/func/Execution_Period_{repeat}/"
            # Make the directory if it does not exist
            if not os.path.exists(dataDir + folder):
                os.makedirs(dataDir + folder)

            filename = dataDir + folder + f"{Config}.nii.gz"
            print(filename)
            # Save the signal to a nifti file
            nib.save(nib.Nifti1Image(condition, filtered_func_file.affine), filename) 



1316 1339
/Users/student/PycharmProjects/data/derivatives/sub-02/func/Execution_Period_0/A1_forw.nii.gz
1339 1366
/Users/student/PycharmProjects/data/derivatives/sub-02/func/Execution_Period_1/A1_forw.nii.gz
1366 1392
/Users/student/PycharmProjects/data/derivatives/sub-02/func/Execution_Period_2/A1_forw.nii.gz
1392 1415
/Users/student/PycharmProjects/data/derivatives/sub-02/func/Execution_Period_3/A1_forw.nii.gz
1415 1430
/Users/student/PycharmProjects/data/derivatives/sub-02/func/Execution_Period_4/A1_forw.nii.gz
896 928
/Users/student/PycharmProjects/data/derivatives/sub-02/func/Execution_Period_0/A1_backw.nii.gz
928 950
/Users/student/PycharmProjects/data/derivatives/sub-02/func/Execution_Period_1/A1_backw.nii.gz
950 970
/Users/student/PycharmProjects/data/derivatives/sub-02/func/Execution_Period_2/A1_backw.nii.gz
970 999
/Users/student/PycharmProjects/data/derivatives/sub-02/func/Execution_Period_3/A1_backw.nii.gz
999 1022
/Users/student/PycharmProjects/data/derivatives/sub-02/func

In [29]:
# # concantenate each repeat's execution period to become one condition for 
# for subject in subjects: 

#     for Config in TaskConfigs:
#         executionPeriod = {} 
#         for repeat in range(5):
#             # Load in the signal file
#             file = dataDir + f'/derivatives/{subject}/func/Execution_Period_{repeat}/{Config}.nii.gz'
#             loaded_file = nib.load(file)
#             loaded_array = loaded_file.get_fdata()
#             print(loaded_array.shape)
#             # executionPeriod[repeat] = nib.load(dataDir + f'/derivatives/{subject}/func/Execution_Period_{repeat}/{Config}.nii.gz')
#             # executionPeriod[repeat] = executionPeriod[repeat].get_fdata()
#             # Concatenate the repeats
#             # executionPeriod = np.concatenate(list(executionPeriod.values()), axis = 3)
#             # Save the the correct directory / subdirectory structure
#             # folder = f"/derivatives/{subject}/func/Execution_Period_All_Repeats/"
#             # # Make the directory if it does not exist
#             # if not os.path.exists(dataDir + folder):
#             #     os.makedirs(dataDir + folder)

#             # filename = dataDir + folder + f"{Config}.nii.gz"
#     break

(108, 108, 64, 23)
(108, 108, 64, 27)
(108, 108, 64, 26)
(108, 108, 64, 23)
(0,)
(108, 108, 64, 32)
(108, 108, 64, 22)
(108, 108, 64, 20)
(108, 108, 64, 29)
(108, 108, 64, 23)
(108, 108, 64, 32)
(108, 108, 64, 27)
(108, 108, 64, 22)
(108, 108, 64, 30)
(108, 108, 64, 22)
(108, 108, 64, 34)
(108, 108, 64, 27)
(108, 108, 64, 28)
(108, 108, 64, 31)
(108, 108, 64, 21)
(108, 108, 64, 25)
(108, 108, 64, 29)
(108, 108, 64, 22)
(108, 108, 64, 19)
(108, 108, 64, 22)
(108, 108, 64, 36)
(108, 108, 64, 24)
(108, 108, 64, 25)
(108, 108, 64, 32)
(108, 108, 64, 21)
(108, 108, 64, 20)
(108, 108, 64, 27)
(108, 108, 64, 31)
(108, 108, 64, 23)
(108, 108, 64, 18)
(108, 108, 64, 31)
(108, 108, 64, 19)
(108, 108, 64, 19)
(108, 108, 64, 25)
(108, 108, 64, 17)
(108, 108, 64, 28)
(108, 108, 64, 27)
(108, 108, 64, 26)
(108, 108, 64, 24)
(108, 108, 64, 23)
(108, 108, 64, 23)
(108, 108, 64, 20)
(108, 108, 64, 25)
(108, 108, 64, 23)
(108, 108, 64, 21)
(108, 108, 64, 18)
(108, 108, 64, 24)
(108, 108, 64, 25)
(108, 1